In [20]:
import yaml
import rospkg

##check
def load_data_from_yaml(known_locations_file='/receptionist_knowledge2.yaml'):
    rospack = rospkg.RosPack()
    file_path = rospack.get_path('config_files') + known_locations_file

    with open(file_path, 'r') as file:
        content = yaml.safe_load(file)
    return content

#check
def save_data_to_yaml(new_knowledge):
    rospack = rospkg.RosPack()
    file_path = rospack.get_path('config_files') + \
        '/receptionist_knowledge2.yaml'
    with open(file_path, 'w') as file:
        documents = yaml.dump(new_knowledge, file)


def get_available_seats():
    data = load_data_from_yaml()
    available_seats = []
    for place, info in data['Places'].items():
        if info['occupied'] == "None":
            available_seats.append(place)
    return available_seats

def assign_seat_to_guest(guest_name, seat):
    data = load_data_from_yaml()
    if seat in data['Places']:
        if data['Places'][seat]['occupied'] is None:
            data['Places'][seat]['occupied'] = guest_name
            data['People'][guest_name]['location'] = seat
            #data['People'][guest_name]['description'] = 'Assigned seat: ' + seat
            return True
        else:
            print(f"Seat {seat} is already occupied.")
            return False
    else:
        return False

def update_seat_assignment(guest_name, new_seat):
    data = load_data_from_yaml()
    if new_seat in data['Places']:
        data['Places'][new_seat]['occupied'] = guest_name
        data['People'][guest_name]['location'] = new_seat
        #data['People'][guest_name]['description'] = 'Assigned seat: ' + new_seat

def process_guests():
    data = load_data_from_yaml()
    for guest, info in data['People'].items():
        if info['location'] == 'waiting':
            available_seats = get_available_seats(data)
            if available_seats:
                seat_assigned = assign_seat_to_guest(data, guest, available_seats[0])
                if seat_assigned:
                    print(f"Assigned seat {available_seats[0]} to {guest}")
                else:
                    print(f"No available seats for {guest}")
            else:
                print(f"No available seats for {guest}")

# Carga los datos desde el archivo YAML
#data = load_data_from_yaml()

# Procesa los invitados y asigna los asientos
#process_guests()

# Actualiza la asignación de asientos si hay un supuesto lugar vacío ocupado
#update_seat_assignment('Michael', 'Place_2')

# Guarda los datos actualizados en el archivo YAML
#save_data_to_yaml(data)


In [21]:
get_available_seats()

['Place_2', 'Place_3']

In [99]:
import yaml
import rospkg
class SCENE():
    def __init__(self, knowlegde_file = '/receptionist_knowledge2.yaml'):
        rospack = rospkg.RosPack()
        self.file_path = rospack.get_path('config_files') + knowlegde_file
        #self.knowledge = self.load_data_from_yaml()
        
        self.last_seat_assigned = "None" #Place_0, Place_1, Place_2
        self.active_guest = "None"       #Guest_0, Guest_1, Guest_2
        
    #-------YAML read and write------------
    def load_data_from_yaml(self):
        with open(self.file_path, 'r') as file:
            data = yaml.safe_load(file)
        return data

    def save_data_to_yaml(self, new_knowledge):
        with open(self.file_path, 'w') as file:
            documents = yaml.dump(new_knowledge, file)
    
    #------Add--------------------
    def add_guest(self, name, drink):
        data = self.load_data_from_yaml()
        guests_len = len(data['People'])
        new_guest = f'Guest_{guests_len}'
        data['People'][new_guest] = {
            'drink': drink, 
            'location': 'None', 
            'name': name, 
            'description': 'None'}
        self.save_data_to_yaml(data)
        self.active_guest = new_guest
        
    def add_description(self, description):
        data = self.load_data_from_yaml()
        data['People'][self.active_guest]['description'] = description
        self.save_data_to_yaml(data)
        
        
    #------Gets-------------------    
    def get_available_seats(self):
        data = self.load_data_from_yaml()
        available_seats = []
        for place, info in data['Places'].items():
            if info['occupied'] == "None":
                available_seats.append(place)
        return available_seats
    
    def get_guest_by_name(self, guest_name):
        data = self.load_data_from_yaml()
        for guest, info in data['People'].items():
            if info['name'] == guest_name:
                return guest
        return 'None'
    
    def get_guest_seat(self, guest_name):
        data = self.load_data_from_yaml()
        for place, info in data['Places'].items():
            if info['occupied'] == guest_name:
                return place
        return 'None'
    
    def get_places_location(self):
        data = self.load_data_from_yaml()
        locs = []
        for place, info in data['Places'].items():
            if place != 'Place_0':
                xyt = []
                xyt.append(info['location']['x'])
                xyt.append(info['location']['y'])
                xyt.append(info['location']['theta'])
                locs.append(xyt)
        return locs
    
    #-----------Seat methods---------------
    def assign_seat_to_guest(self, seat):
        data = self.load_data_from_yaml()
        #guest = self.get_guest_by_name(guest_name)
        guest = self.active_guest
        if guest != 'None':
            guest_name = data['People'][guest]['name']
            data['Places'][seat]['occupied'] = guest_name
            data['People'][guest]['location'] = seat
            self.save_data_to_yaml(data)
            return True
        else:
            return False
        
    def update_seat_assignment(self, guest_name, new_seat):
        data = self.load_data_from_yaml()
        place = self.get_guest_seat(guest_name)
        guest = self.get_guest_by_name(guest_name)
        if place != 'None' and guest != 'None':
            data['Places'][place]['occupied'] = 'None'
            data['Places'][new_seat]['occupied'] = guest_name
            data['People'][guest]['location'] = new_seat
            self.save_data_to_yaml(data)
            return True
        else:
            return False
    
    def clean_knowledge(self, host_name, host_location):
        data = self.load_data_from_yaml()
        host = data['People']['Guest_0']
        host['name'] = host_name
        host['location'] = host_location
        data['People'] = {'Guest_0': host}
        for place, info in data['Places'].items():
            if place == host_location:
                info['occupied'] = host_name
            elif place == 'Place_0':
                info['occupied'] = 'Not available'
            else:
                info['occupied'] = 'None'
        self.save_data_to_yaml(data)
        


In [100]:
party = SCENE()

In [101]:
party.clean_knowledge('ruben', 'Place_1')

In [102]:
party.add_guest('george', 'water')

In [103]:
party.add_description('Good friend')

In [110]:
party.get_available_seats()

['Place_2']

In [105]:
party.get_guest_by_name('oscar')

'None'

In [106]:
party.get_guest_seat('george')

'None'

In [107]:
party.get_places_location()

[[10.05, -2.15, 0.05], [10.14, -3.08, 0.15], [9.52, -3.18, 4.8]]

In [112]:
seats = party.get_available_seats()
party.assign_seat_to_guest(seats[0])

True

In [109]:
party.update_seat_assignment('george', 'Place_3')

True

In [111]:
party.add_guest('michael', 'wine')

In [114]:
len(party.get_available_seats())

0

In [118]:
party.get_available_seats()[0]

IndexError: list index out of range